In [7]:
import os
from utils import create_directory, interpolate_image
import subprocess
import cv2
import albumentations as A
import numpy as np
from config import *

In [2]:
BASE_DIR = "DATA"
original_data = f"{BASE_DIR}/actual_data"
create_directory([BASE_DIR,"train_data"])
create_directory([BASE_DIR,"test_data"])
training_images = create_directory([BASE_DIR,"train_data","images"])
training_masks = create_directory([BASE_DIR,"train_data","masks"])
create_directory([BASE_DIR,"test_data","images"])
create_directory([BASE_DIR,"test_data","masks"])
create_directory([BASE_DIR,"augmented"])
augmented_images = create_directory([BASE_DIR,"augmented","images"])
augmented_masks = create_directory([BASE_DIR,"augmented","masks"])
compressed = create_directory([BASE_DIR,"compressed"])

DATA/train_data already existed
DATA/test_data already existed
DATA/train_data/images already existed
DATA/train_data/masks already existed
DATA/test_data/images already existed
DATA/test_data/masks already existed
DATA/augmented already existed
DATA/augmented/images already existed
DATA/augmented/masks already existed
DATA/compressed already existed


In [3]:
subprocess.call(f"convert {original_data}/training.tif {BASE_DIR}/train_data/images/train-%04d.png", shell=True)
subprocess.call(f"convert {original_data}/testing.tif {BASE_DIR}/test_data/images/test-%04d.png", shell=True)
subprocess.call(f"convert {original_data}/training_groundtruth.tif {BASE_DIR}/train_data/masks/train-%04d.png", shell=True)
subprocess.call(f"convert {original_data}/testing_groundtruth.tif {BASE_DIR}/test_data/masks/test-%04d.png", shell=True)

0

In [4]:
transform_crop = A.Compose([
    A.RandomCrop(width=input_images_size,height=input_images_size,p=1,always_apply=True),
    A.HorizontalFlip(p=1,always_apply=True),
])
transform_rotate90 = A.Compose([
    A.RandomRotate90(p=1,always_apply=True)
])
transform_centercrop = A.Compose([
    A.CenterCrop(width=input_images_size,height=input_images_size,p=1,always_apply=True)
])
transform_noise = A.Compose([
    A.RandomBrightnessContrast(p=1,always_apply=True)
])
transform_all = A.Compose([
    A.RandomCrop(width=input_images_size,height=input_images_size,p=1,always_apply=True),
    A.HorizontalFlip(p=1,always_apply=True),
    A.RandomRotate90(p=1,always_apply=True),
    A.RandomBrightnessContrast(p=1,always_apply=True)
])

In [5]:
for e in os.listdir(training_images):
    image = cv2.imread(f"{training_images}/{e}",0)
    mask = cv2.imread(f"{training_masks}/{e}",0)
    mask[mask==255]=1
    cv2.imwrite(f"{augmented_images}/00_{e}", image)
    cv2.imwrite(f"{augmented_masks}/00_{e}", mask)
    
    transformed = transform_rotate90(image=image, mask=mask)
    cv2.imwrite(f"{augmented_images}/01_{e}", transformed["image"])
    cv2.imwrite(f"{augmented_masks}/01_{e}", transformed["mask"])
    
    transformed = transform_crop(image=image, mask=mask)
    cv2.imwrite(f"{augmented_images}/02_{e}", transformed["image"])
    cv2.imwrite(f"{augmented_masks}/02_{e}", transformed["mask"])
    
    transformed = transform_all(image=image, mask=mask)
    cv2.imwrite(f"{augmented_images}/03_{e}", transformed["image"])
    cv2.imwrite(f"{augmented_masks}/03_{e}", transformed["mask"])

In [6]:
for e in os.listdir(augmented_images):
    image = cv2.imread(f"{augmented_images}/{e}",0)
    mask = cv2.imread(f"{augmented_masks}/{e}",0)
    
    image = np.array(image, dtype="float32")
    image = interpolate_image(image, target_shape)
    mask = mask.astype("uint8")
    mask = interpolate_image(mask, target_shape)
    mask[mask==255]=1
    np.savez_compressed(f"{compressed}/{e}",
                            image=image,
                            mask=mask)